In [1]:
import pickle
import flask
import pandas as pd
import numpy as np
import os, io, json
import torch
import torch.nn as nn
import torch.nn.functional as F
from math import sin, cos, sqrt, atan2, radians

In [2]:
class CNN_RS_FM(torch.nn.Module):
    def __init__(self, poolingFunction, bnFunction, activationFunction, kernel=None, stride=None, padding=None):
        super(CNN_RS_FM, self).__init__()

        # building network with specified number of layers, poolingFunction and bnFunction
        self.network = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=2),
            poolingFunction(kernel_size=2, stride=2, padding=1),
            activationFunction(),
            # bnFunction(num_features=16),
        
            torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            poolingFunction(kernel_size=2, stride=2, padding=1),
            activationFunction(),
            # bnFunction(num_features=32),

            torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            poolingFunction(kernel_size=2, stride=2, padding=1),
            activationFunction(),
            # bnFunction(num_features=64),
        )

        self.linStack = torch.nn.Sequential(
            torch.nn.Linear(in_features=1344, out_features=672),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(num_features=672),

            torch.nn.Linear(in_features=672, out_features=336),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(num_features=336),

            torch.nn.Linear(in_features=336, out_features=7),
        )

    def forward(self, x):
        x = self.network(x)
        x = torch.nn.Flatten(1)(x)
        x = self.linStack(x)
        return x

In [3]:
def prep_RS_FM(df, samplesize:int=43):
  '''
  Prepares data for Group RS_FM

  - removes all columns but ['acc_x','acc_y','acc_z','mag_x','mag_y','mag_z','gyr_x','gyr_y','gyr_z','ori_x','ori_y','ori_z','ori_w']
  - turns pd.DataFrame into np.array
  - splits array into samples of size samplesize
  - turns list with splits into array
  - checks shape
  - turns array into tensor

  '''
  # remove unnecessary columns
  df.columns = ['time','name','activity','acc_x','acc_y','acc_z','mag_x','mag_y','mag_z','gyr_x','gyr_y','gyr_z','ori_x','ori_y','ori_z','ori_w','lat','long']
  df.loc[:, 'time'] = pd.to_datetime(df.loc[:, 'time'] + "000") # format="%d.%m.%Y %H:%M.%S.%f"
  df = df[['acc_x','acc_y','acc_z','mag_x','mag_y','mag_z','gyr_x','gyr_y','gyr_z','ori_x','ori_y','ori_z','ori_w']]
  arr = df.values

  # split into samples
  split = arr.shape[0] // samplesize
  stop = samplesize * split
  arr = arr[:stop,:]
  arr = np.array(np.array_split(arr, split))

  # assert shape
  assert arr.shape == (split, samplesize, 13)

  # turn into tensor
  arr = torch.from_numpy(arr)#.to(device)

  return arr

In [5]:
def pred_CNN_RS_FM(data:torch.tensor, model):
  pred = model(data)
  pred = torch.argmax(pred, dim=1)
  

In [117]:
model = CNN_RS_FM(poolingFunction=torch.nn.MaxPool2d, bnFunction=torch.nn.BatchNorm2d, activationFunction=torch.nn.ReLU)
model.load_state_dict(torch.load("models/97%model.pt", map_location=torch.device('cpu')))
test_df = pd.read_csv('../df_clean.csv').iloc[:,1:]
test_df = prep_RS_FM(df=test_df)

In [133]:
model.eval()
pred = model(test_df[0:4].unsqueeze(1).float())
pred = torch.argmax(pred, dim=1)
pred = torch.argmax(pred).item()
labels = ['Sitting', 'Transport', 'Bicycling', 'Walking', 'Elevatoring', 'Jogging', 'Stairway']
labels[pred]

'Sitting'